# Wilson - Cowan whole brain simulation

To model the coarse grained activity of large neuronal populations so called neural mass models or mean field have been used since the 1970s. They are typically cast as systems of ordinary differential equations (ODEs) for neural activity evolution. The one of the most used  is the Wilson-Cowan and related equations [Wilson, H., and Cowan, J. (1972)](https://doi.org/10.1016/S0006-3495(72)86068-5) They were able to model many elements of observed neural phenomena [(Chow & Karimipanah, 2020)](https://journals.physiology.org/doi/full/10.1152/jn.00404.2019).   The original equations refer to two-dimensional model that aim to describe activity of  two populations of neurons, inhibitory and excitatory populations. There are several modifications of activity equations, here we use the modification described in paper inhibitory and excitatory populations [(Papadopoulos et al., 2020)](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1008144). The dynamics of the $j$ brain area described with coupled differential equations:

$$\tau_{E} \dot{E_i} = - E_i + (1- E_i)f_{E}(c_{EE}*E_i - c_{IE}*I_i+K_{gl}\sum_j C_{ij}E_{j-delay_{ij}}+PE_i) + OU_{i}$$

$$\tau_{I} \dot{I_i} = - I_i + (1- I_i)f_{I}(c_{EI}*E_i-c_{II}*I_i+PI_i) + OU_{i}$$


$$f_E=\frac{1.0} {(1 + \exp(-a_{exc} * (x - \mu_{exc}))}$$
$$f_I=\frac{1.0} {(1 + \exp(-a_{inh} * (x - \mu_{inh}))}$$


The variables $E$ and $I$ corresponds to firing rates of excitatory and inhibitory populations of region j, and $\tau_E$ and $\tau_I$ are the excitatory and inhibitory time constants, respectively. $f_E$ and $f_I$ are the non-linear response (activation) functions  with the $a_{exc}$ and $a_{inh}$ - excitatory and inhibitory gain, and $\mu_{exc}$ excitatory and $\mu_{inh}$ inhibitory firing threshold.


Dynamics of the both populations in region $i$ are driven by the next sources:
 - local interaction between excitatory and inhibitory populations inside the one region, i.e. local E-E($c_{EE}$), E-I ($c_{EI}$), I-E ($c_{IE}$), I-I ($c_{II}$) coupling  
  - long-range excitation from other populations ($C_{ij}$)
  - non-specific background drive $PE_i$
  - noise $OU$ (here modelled with Ornstein-Uhlenbeck process)
  
  
Furthermore, $K_{gl}$ is a global coupling that tunes the overall interaction strength between different brain areas, and $delay_{ij}$ is a time delay between regions $i$ and $j$ that arises due to the spatial embedding of the brain network   


Usually, anatomical structural connectivity  matrix used as long-range coupling  matrics $C$...
=== And our wonderfull approach with synaptic weights comes here .....

In [1]:
from tqdm import tqdm
import xarray as xr
import os
import seaborn as sns
import sys
sys.path.append('..')
from scipy import signal, stats, io
from neurolib.models.wc import WCModel
from neurolib.models import bold
from neurolib.utils import functions
from task_fc_simulation.read_utils import read_onsets_from_input, read_generate_task_matrices
from task_fc_simulation.onset_design_model import WCOnsetDesign, HRF
from task_fc_simulation.weight_matrix_utils import normalize, generate_modulars
import numpy as np
import gc
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['font.size'] = 10
plt.rcParams['image.cmap'] = 'plasma'
np.set_printoptions(suppress=True)
%load_ext autoreload
%autoreload 2

## Setting up WC model parameters 

We need to set up model parameters. Default values used in neurolib borrowed from [(Papadopoulos et al., 2020)](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1008144). But optionally all these values could be modified. 

![Here](https://journals.plos.org/ploscompbiol/article/figure/image?size=large&id=10.1371/journal.pcbi.1008144.t001)

***setting up wilson-cowan model parameters***

In [2]:
wc_params = {'exc_ext': 0.758, # baseline external input to E
             'K_gl': 2.63, # global coupling strength
             'sigma_ou': 3.5 * 1e-3, # noise intensity
             'inh_ext': 0, # baseline external input to I
             'tau_ou': 5, # ms Timescale of the Ornstein-Uhlenbeck noise process
             'a_exc': 1.5, # excitatory gain
             'a_inh': 1.5, # inhibitory gain
             'c_excexc': 16, #local E-E coupling
             'c_excinh': 15, # local E-I coupling
             'c_inhexc': 12, # local I-E coupling
             'c_inhinh': 3, # local I-I coupling
             'mu_exc': 3,  # excitatory firing threshold
             'mu_inh': 3,  # inhibitory firing threshold
             'tau_exc': 2.5, # excitatory time constant
             'tau_inh': 3.75, # inhibitory time constant
             'signalV': 10 # signal transmission speed between areas
            }

## Download design info 
if option SMALL is used  - the simulation will be on the smallest dataset for testing purposes

In [3]:
SMALL = False
mat_path = '../data/27_EVENT_[700ms_TR]_[500ms_DUR]_[6s_ISI]_[100_TRIALS].mat'
input_data = io.loadmat(mat_path)
if SMALL:
    small_input_data = input_data.copy()
    small_input_data['onsets'][0,0] = input_data['onsets'][0,0][:4]
    small_input_data['onsets'][0,1] = input_data['onsets'][0,1][:4]
    small_mat_path = os.path.join(os.path.dirname(mat_path),'small_'+os.path.basename(mat_path))
    io.savemat(small_mat_path, small_input_data)
    mat_path = small_mat_path
print(f"In the simulation {os.path.basename(mat_path)} will be used")    

In the simulation 27_EVENT_s_TR]_s_DUR]_[6s_ISI]_[100_TRIALS].mat will be used


## Synaptic weight generation

In [4]:
N_ROIs = 100
SIGMA = 0.1
NORM_TYPE = "cols" #cols, raw
NUM_MODULES=4
#generate matrices with the coefficients, which we defined in "task_matrices" and "rest_matrices" fields of the mat file
C_rest, C_task_dict = read_generate_task_matrices(mat_path, N_ROIs, num_modules=NUM_MODULES,
                                sigma=SIGMA, norm_type=NORM_TYPE)
C_task_list = list(C_task_dict.values())
C_list = [C_rest]+C_task_list
title_list =['Rest']+ list(C_task_dict.keys())
fig, axs = plt.subplots(1, 3, figsize = (15,4))

#list of possible colormaps here: https://matplotlib.org/stable/tutorials/colors/colormaps.html
for i in [0,1,2]:
    im = axs[i].imshow(C_list[i], cmap='jet', vmin = 0, vmax=0.05); axs[i].set_title(title_list[i]);
    fig.colorbar(im, ax = axs[i], fraction=0.046, pad=0.04);
fig.tight_layout()

In [5]:
C_task_dict["Rest"] = C_rest
io.savemat("../data/SynapticWeights.mat", C_task_dict)

## Simulations part

### Simulation setting

Here we need to define simulation regime parameters.
 - :: delay (int or np.array of ints) - delay parameter or delay matrix in number of steps, default = 50
 - :: rest_before (True, False) - if we need to generate rest session before zero time moment 
 - :: first_duration (default 12)  - duration of first rest simulation
 - :: last_duration (default 8) - duration of rest block at the end of session
 - :: TR (default 2) - repetition time for bold resampling (in seconds)
 - :: activity (bool) - if True original neural activity will be saved with a_s_rate 
 - :: a_s_rate (default 0.02) - neuronal activity sampling rate (in seconds)
 

In [6]:
sim_parameters = {"delay": 250,"rest_before":True, "first_duration": 10, "last_duration": 20}
TR = 0.7
a_s_rate = 5*1e-3 #sampling in s, original integration equal to 0.1 ms or 0.0001s
activity = True
#see notebook HRFConvolution for parameters description
bw_params = {"rho": 0.34, "alpha": 0.32, "V0": 0.02, "k1_mul": None,
                     "k2": None, "k3_mul": None, "gamma": None, "k": None, "tau":None}



In [7]:
setting_dict = {**wc_params, **sim_parameters, **{ key:value for (key,value) in bw_params.items() if value is not None}, 'a_s_rate': a_s_rate, 'TR': TR}
resmatfile = os.path.join(os.path.dirname(mat_path),'SIM_SET_'
                        +os.path.basename(mat_path))
io.savemat(resmatfile, setting_dict)
#io.loadmat(resmatfile)

In [ ]:
%%time
#path to mat file with the predefined structure with onsets and synaptic weights matrices parameters
wc_block = WCOnsetDesign.from_matlab_structure(mat_path, sigma=SIGMA, norm_type=NORM_TYPE, num_modules=NUM_MODULES, 
                                               num_regions= N_ROIs, **wc_params, **sim_parameters)
wc_block.generate_full_series(TR=TR, activity=activity, a_s_rate=a_s_rate, clear_exc = False, **bw_params)

In [ ]:
wc_block.BOLD.shape

## Bold:: external task activation

Hemodynamic Signal = WCO-scaled + SF-loc * TaskDesign⊗HRF

In the input file we have parameters with additional local activation for each block, two arrays with the dimension equal to NUM_MODULES:
 - if 1 in first and 0 in second : only taskA
 - if 1 in first and 1 in second : taskB+taskA
 - if 0 in first and 1 in second : only taskB

so we need to create onset list for all these blocks


In [ ]:
print("Additional activation: ",input_data["activations"])
t_coactiv, coactiv, bold_coactiv = wc_block.generate_local_activations(mat_path, act_scaling=0.5, **bw_params)

In [ ]:
print("Additional activation: ",input_data["activations"])
fig, axs = plt.subplots(1, 3, figsize = (15,4))
node_ids = [0,10,20]
draw_last = 100
draw_last_TR = int(draw_last/TR) #in seconds
for i in range(3):
        
    axs[i].plot(t_coactiv[-draw_last_TR:]/1e3, 0.03*coactiv[node_ids[i],-draw_last_TR:]);
    axs[i].plot(t_coactiv[-draw_last_TR:]/1e3, bold_coactiv[node_ids[i],-draw_last_TR:]); axs[i].set_title(f"Bold and resampled act for {node_ids[i]} node");

##  Results: properties investigation
### Neuronal activity

In [ ]:
# results verification: power spectrum original acitivity and resampled part for the final resting state 
fig, axs = plt.subplots(1, 2, figsize = (10,4))
maxfr = 100
f, Pxx_Welsh =  functions.getMeanPowerSpectrum(wc_block.wc.exc,0.1, spectrum_windowsize=1, maxfr=maxfr)
axs[0].semilogy(f,  Pxx_Welsh); axs[0].set_title('Original activity (dt=0.1ms)', fontsize=12);
f, Pxx_Welsh =  functions.getMeanPowerSpectrum(wc_block.activity["series"][:,-800:],5, spectrum_windowsize=1, maxfr=maxfr)
axs[1].semilogy(f,  Pxx_Welsh, label="Rest(last 4 sec)"); axs[1].set_title('Resampled activity (dt=5 ms)', fontsize=12);
fig.suptitle("Mean Power density for excitation activity", fontsize=16); 
fig.tight_layout(rect=[0, 0.03, 1, 0.95])
f, Pxx_Welsh =  functions.getMeanPowerSpectrum(wc_block.activity["series"],5, spectrum_windowsize=1, maxfr=maxfr)
axs[1].semilogy(f,  Pxx_Welsh, label="Full data");
axs[1].legend();

In [ ]:
results_dict = {"PowerDensity": np.array([f, Pxx_Welsh])}
io.savemat(resmatfile, {**results_dict, **setting_dict})

In [ ]:
resmatfile

In [ ]:
#io.loadmat(resmatfile)

## Phase synhronization for neural actvity
Phase synchronization measured as intersite phase clustering. Phase-based connectivity analyses rely on the distribution
of phase angle differences between two electrodes, with the idea that when neural
populations are functionally coupled, the timing of their oscillatory processes, as measured
through phase, becomes synchronized.

$$ISPC_f= \frac{1}{n}\sum_1^n \exp^{i(\phi_x-\phi_y)}$$

Phase angles are obtained by a point in complex space resulting from the convolution between a complex wavelet and the
data or from applying the Hilbert transform to bandpass-filtered data (here we used the hilbert transform).


In [ ]:
ds = wc_block.compute_phase_diff(low_f=38, high_f=42, return_xr=True)
print("Results as xarray dataset")

In [ ]:
def mean_abs(x, dim=None):
    return abs(np.exp(1j * x).mean(dim=dim))
tasks = ['Rest','A','B']
N_ROIs = ds.dims['region']
title_list = ['PLV_Rest','PLV_A', 'PLV_B']
roi_idx1, roi_idx2 = np.triu_indices(N_ROIs,k=1)
fig, axs = plt.subplots(1, 3, figsize = (15,4))
PLV_dict_keys = ['PLV_REST','PLV_A', 'PLV_B']
PLV_dict = {}
#take a lot of time to compute matrix, so we can take smaller subset
SUBSET = True
last_time = 120 #in m/s
for i in [0,1,2]:
    if SUBSET:
        synh = ds.sel(time=slice(0,last_time*1000)).groupby('task_type')[tasks[i]].map(mean_abs, dim = 'time').phase_diff.data
    else:
        synh = ds.groupby('task_type')[tasks[i]].map(mean_abs, dim = 'time').phase_diff.data
    sym_synh = 0.3*np.eye(N_ROIs)
    sym_synh[roi_idx1, roi_idx2] = synh
    matx = sym_synh+sym_synh.T
    im = axs[i].imshow(matx, cmap='jet', vmin = 0, vmax=1); axs[i].set_title(title_list[i]);
    PLV_dict[PLV_dict_keys[i]]= matx
    
    fig.colorbar(im, ax = axs[i], fraction=0.046, pad=0.04);
fig.tight_layout()

In [ ]:
io.savemat(resmatfile, {**results_dict, **setting_dict, **PLV_dict})

## Envelope 
Raw signal plus envelope and bold signal

In [ ]:
env_dict = wc_block.draw_envelope_bold_compare(node_id=26, low_f=38, high_f=42, low_pass=None, 
                               drop_first_sec=10, shift_sec = 3.7,  plot_first = 120)

In [ ]:
io.savemat(resmatfile, {**results_dict, **setting_dict, **PLV_dict, **env_dict})

In [ ]:
resmatfile

##  Generate BOLD for N_subj and save to MAT FILE

In [ ]:
N_ROIs, mat_path

In [ ]:
%%time
N_subj=100
C_control = []
BOLD_subj = np.zeros((*wc_block.BOLD.transpose().shape,N_subj))
for i in tqdm(range(N_subj)):
    wc_block = WCOnsetDesign.from_matlab_structure(mat_path, sigma=SIGMA, norm_type=NORM_TYPE, num_regions= N_ROIs,
                                                   **wc_params, **sim_parameters)
    wc_block.generate_full_series(TR=TR, activity=False, a_s_rate=a_s_rate, **bw_params)
    BOLD_subj[:,:,i] = wc_block.BOLD.transpose()
    C_control.append([wc_block.C_rest, wc_block.C_task_dict])

In [ ]:
boldmatfile = os.path.join(os.path.dirname(mat_path),'SIM_BOLD_'
                        +os.path.basename(mat_path))
bold_dict = {'oscill': BOLD_subj, 'coact': bold_coactiv.transpose()}
io.savemat(boldmatfile, bold_dict)

## Parameter exploration 

4 min for 100 nodes

In [8]:
sigma_ou_list = [4.3 * 1e-3]
ext_ext_list = [0.755, 0.756, 0.757, 0.758, 0.759, 0.760, 0.761, 0.762, 0.763, 0.764, 0.765]
k_gl_list = [2.60, 2.61, 2.62, 2.63, 2.64, 2.65, 2.66, 2.67, 2.68, 2.69, 2.70, 2.71, 2.72, 2.73, 2.74, 2.75]

In [9]:
def plot_cors(bold, title_list=None, masks=None, noise_level=0.5, to_plot=False):
    
    bold = bold[3:,:]
    
    if title_list is None:
        title_list = ["Rest", "TaskA", "TaskB", "TaskA-TaskB"]

    if masks is None:    
        maskRest = np.array(bold.shape[0] * [False])
        maskTaskB = np.array(bold.shape[0] * [False])
        maskTaskA  = np.array(bold.shape[0] * [False])
        badMask = np.array(bold.shape[0] * [False])
        for el in np.nditer(input_data['onsets'][0, 0]//2):
            maskTaskA[el+3:el+10]=True
            maskRest[el+13:el+20]=True
        
        for el in np.nditer(input_data['onsets'][0, 1]//2):
            maskTaskB[el:el+10]=True 
            maskRest[el+13:el+20]=True

        masks = [maskRest, maskTaskA, maskTaskB]
        

    BoldNormed = (bold - bold.mean()) / (bold.std())
    BoldNoised = BoldNormed + noise_level * np.random.normal(size=BoldNormed.shape) 
    corrs = [np.corrcoef(BoldNoised[masks[i],:].T)  for i in range (3)]
    corrs.append(corrs[1] - corrs[2])

    if to_plot:

        fig, axs = plt.subplots(1, 4, figsize = (15,5))

        for i in [0,1,2]:
            im = axs[i].imshow(corrs[i], cmap='jet', vmin = 0, vmax=1); axs[i].set_title(title_list[i]);
            fig.colorbar(im, ax=axs[i], fraction=0.046, pad=0.04);
        im = axs[3].imshow(corrs[3], cmap='jet', vmin=-1, vmax=1); axs[3].set_title(title_list[3]);
        fig.colorbar(im, ax=axs[3], fraction=0.046, pad=0.04);    
        fig.suptitle(f"Correlation structure with NSR = {noise_level}", fontsize=14)    
        fig.tight_layout()
        
    return corrs

In [10]:
dict_noise0 = {}
dict_noise1 = {}
dict_with_rcoeff = {}

N_ROIs = 100

for sigm in sigma_ou_list:
    for k in k_gl_list:
        for ext in ext_ext_list:
            
            wc_block = WCOnsetDesign.from_matlab_structure(mat_path, sigma=SIGMA, norm_type=NORM_TYPE, num_modules=4, 
                                                           num_regions=N_ROIs, **wc_params, **sim_parameters)
            
            wc_block.wc.params['sigma_ou'] = sigm
            wc_block.wc.params['K_gl'] = k
            wc_block.wc.params['exc_ext'] = ext
            
            wc_block.generate_full_series(TR=TR, activity=activity, a_s_rate=a_s_rate, **bw_params)

            corrs_noise0 = plot_cors(wc_block.BOLD.transpose(), noise_level=0, to_plot=False)
            corrs_noise1 = plot_cors(wc_block.BOLD.transpose(), noise_level=1, to_plot=False)


            dict_noise0[(sigm, k, ext)] = corrs_noise0
            dict_noise1[(sigm, k, ext)] = corrs_noise1
            
            r_max = []
            for node in range(100):
                output_dict = wc_block.draw_envelope_bold_compare(
                                    node_id=node, low_f=38, high_f=42, low_pass=None, 
                                    drop_first_sec=9, shift_sec=4,  plot_first=3, to_plot=False)
                #plt.close()
                rcoeff_list = output_dict['rs'][1]
                r_max.append(max(rcoeff_list))
                
                del output_dict


            dict_with_rcoeff[(sigm, k, ext)] = [np.max(r_max), np.mean(r_max)]
            del wc_block; gc.collect()

In [11]:
def plot_heatmap1(fix='sigma_ou', value=0.00425, mean_or_max='max'):
    d = {'sigma_ou': 0,
        'k_gl': 1,
        'ext_ext': 2}
    m = {'max': 0,
         'mean': 1}
    
    coef = [i for i in dict_with_rcoeff if i[d[fix]] == value]
    d.pop(fix, None)
    x = d[list(d.keys())[0]]
    y = d[list(d.keys())[1]]

    x_list, y_list = [], []
    for i in coef:
        x_list.append(i[x])
        y_list.append(i[y])

    x_list, y_list = np.unique(x_list), np.unique(y_list)

    rrr = np.zeros((len(x_list), len(y_list)))
    for en, i in enumerate(x_list):
        coef2 = [c for c in coef if c[x] == i]
        for ej, j in enumerate(coef2):
            rrr[en, ej] = dict_with_rcoeff[j][m[mean_or_max]]
    

    sns.heatmap(rrr, annot=True, xticklabels=y_list, yticklabels=x_list).set(
        title=f'fix {fix}={value}')

    return rrr

In [12]:
def plot_heatmap2(data, fix='ext_ext', value=0.76, condition='Rest'):

    """ data: dict_noise1 или dict_noise0

        condition: 'Rest'
                   'Task_A'
                   'Task_B'
                   'Task_A-Task_B' """

    d = {'sigma_ou': 0,
        'k_gl': 1,
        'ext_ext': 2}

    m = {'Rest': 0,
         'Task_A': 1,
         'Task_B': 2,
         'Task_A-Task_B': 3}

    fixed_matr = [i for i in data if i[d[fix]] == value]
    d.pop(fix, None)
    x = d[list(d.keys())[0]]
    y = d[list(d.keys())[1]]
    
    x_list, y_list = [], []
    for i in fixed_matr:
        x_list.append(i[x])
        y_list.append(i[y])
    
    x_list, y_list = np.unique(x_list), np.unique(y_list)
    
    rrr = np.zeros((len(x_list), len(y_list)))
    for en, i in enumerate(x_list):
        coef2 = [c for c in fixed_matr if c[x] == i]
        
        for ej, j in enumerate(coef2):
            mat1 = data[j][m[condition]].flatten()

            if condition == 'Rest':
                mat2 = C_rest.flatten()
                rrr[en, ej] = np.corrcoef(mat1, mat2)[0, 1]
            elif condition == 'Task_A-Task_B':
                mat2 = C_task_dict['Task_A'] - C_task_dict['Task_B']
                rrr[en, ej] = np.corrcoef(mat1, mat2.flatten())[0, 1]
            else:
                mat2 = C_task_dict[condition].flatten()
                rrr[en, ej] = np.corrcoef(mat1, mat2)[0, 1]
    
    sns.heatmap(rrr, annot=True, xticklabels=y_list, yticklabels=x_list, fmt='.4g').set(
        title=f'fix {fix}={value}, {condition}')
    plt.ylabel(f'{list(d.keys())[0]}')
    plt.xlabel(f'{list(d.keys())[1]}');

    return rrr

In [13]:
plot_heatmap2(dict_noise0, fix='sigma_ou', value=4.3* 1e-3, condition='Task_A-Task_B');

In [14]:
_=plot_heatmap1('sigma_ou', value=4.3 * 1e-3, mean_or_max='mean')

In [15]:
from scipy.io import savemat

for i in dict_with_rcoeff:
    io.savemat(f'C:\\TMFC_SIM\\WCTaskFcSimulation\\data\\Second_pass\\rcoeff_{str(i)}.mat', 
               {'max': dict_with_rcoeff[i][0], 
                'mean': dict_with_rcoeff[i][1]})


names = ["Rest", "TaskA", "TaskB", "TaskA_TaskB"]
dict_noise0_new = {}
for i in dict_noise0:
    for t in range(4):
        dict_noise0_new[names[t]] = dict_noise0[i][t]
    io.savemat(f'C:\\TMFC_SIM\\WCTaskFcSimulation\\data\\Second_pass\\noise0_{i}.mat', dict_noise0_new)


dict_noise1_new = {}
for i in dict_noise1:
    for t in range(4):
        dict_noise1_new[names[t]] = dict_noise1[i][t]
    io.savemat(f'C:\\TMFC_SIM\\WCTaskFcSimulation\\data\\Second_pass\\noise1_{i}.mat', dict_noise1_new)